In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/kermany2018"))

# Any results you write to the current directory are saved as output.

In [ ]:
Training_Dir = '../input/kermany2018/oct2017/OCT2017 /train'
Validation_Dir = '../input/kermany2018/oct2017/OCT2017 /val'
Testing_Dir = '../input/kermany2018/oct2017/OCT2017 /test'

In [ ]:
from tensorflow.keras.applications.xception import Xception
pre_trained_model = Xception(input_shape = (150, 150, 3), weights = None,include_top = False)
pre_trained_model.summary()

In [ ]:
weights_file = '../input/keras-pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model.load_weights(weights_file)

In [ ]:
#for layer in pre_trained_model.layers:
#    layer.trainable = False
len(pre_trained_model.layers)

In [ ]:
last_layer = pre_trained_model.get_layer('block14_sepconv2_act')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (4, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

training_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale = 1./255)
testing_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	Training_Dir,
	target_size=(150,150),
  shuffle=True,
	class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
	Validation_Dir,
	target_size=(150,150),
  shuffle=True,
	class_mode='categorical'
)

testing_generator = testing_datagen.flow_from_directory(
	Testing_Dir,
	target_size=(150,150),
  shuffle=True,
	class_mode='categorical'
)

In [ ]:
import numpy as np
np.unique(train_generator.classes)

In [ ]:
train_generator.class_indices, validation_generator.class_indices, testing_generator.class_indices

In [ ]:
np.unique(train_generator.classes, return_counts = True)

In [ ]:
np.unique(testing_generator.classes, return_counts = True)

In [ ]:
history = model.fit_generator(train_generator, epochs=3, validation_data = testing_generator, verbose = 1)

In [ ]:
history = model.fit_generator(train_generator, epochs=12, validation_data = testing_generator, verbose = 1)

In [ ]:
history.history['acc']

In [ ]:
history.history['val_acc']

In [ ]:
history.history['loss']

In [ ]:
history.history['val_loss']

In [ ]:
training_datagen = ImageDataGenerator(rescale = 1./255.)

train_generator = training_datagen.flow_from_directory(
	Training_Dir,
	target_size=(150,150),
  shuffle=True,
	class_mode='categorical'
)

In [ ]:
y_train_pred = model.predict_generator(train_generator)
y_train = train_generator.classes[train_generator.index_array]
y_train_pred = np.argmax(y_train_pred, axis=-1)
from sklearn.metrics import confusion_matrix
import itertools
cnf_matrix = confusion_matrix(y_train, y_train_pred)
cnf_matrix

In [ ]:
from sklearn.metrics import classification_report
target_names = ['CNV', 'DME', 'DRUSEN', 'NORMAL']
print(classification_report(y_train, y_train_pred, target_names=target_names))

In [ ]:
y_test_pred = model.predict_generator(testing_generator)
y_test = testing_generator.classes[testing_generator.index_array]
y_test_pred = np.argmax(y_test_pred, axis=-1)
cnf_matrix = confusion_matrix(y_test, y_test_pred)
cnf_matrix

In [ ]:
print(classification_report(y_test, y_test_pred, target_names=target_names))